In [1]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from torch import nn
import os
import json
import time
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from h3 import h3
from geopy.distance import geodesic as vincenty
from tqdm import tqdm

class MultiTaskModel(nn.Module):
    def __init__(self, model_name, num_labels_res8, num_labels_res10, num_labels_res9):
        super(MultiTaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier_res8 = nn.Linear(self.hidden_size, num_labels_res8)
        self.classifier_res10 = nn.Linear(self.hidden_size, num_labels_res10)
        self.classifier_res9 = nn.Linear(self.hidden_size, num_labels_res9)
    
    def forward(self, input_ids, attention_mask=None, labels_res8=None, labels_res10=None, labels_res9=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        logits_res8 = self.classifier_res8(pooled_output)
        logits_res10 = self.classifier_res10(pooled_output)
        logits_res9 = self.classifier_res9(pooled_output)

        return {"logits_res10": logits_res10, "logits_res8": logits_res8, "logits_res9": logits_res9}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
address_data_path = '/home/uddeshya.singh/DATA/PUNJAB_test/PUNJAB_with_grid_ids_test.csv' 
fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/level-two_models/Fine-tuned-save'
model_dir =  "/home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir"

# Load configuration
with open(os.path.join(fine_tuned_lm_trained_save_dir, 'config.json'), 'r') as f:
    config = json.load(f)

num_labels_res8 = config['num_labels_res8']
num_labels_res10 = config['num_labels_res10']
num_labels_res9 = config['num_labels_res9']


In [16]:
df = pd.read_csv(address_data_path)

In [17]:
df.head()

addr_hash  \
0  c5cf1f7511c2ae3e385bf9f8f846317c   
1  b5a0c40241038bd9d410a91df433926b   
2  3104b020a7131e005a5c68a87ae18dd4   
3  04aab31bb493bc0fca2619c29fdbc894   
4  70126c55ada9a880de480a923e237cf7   

                                     cleaned_address  lookup_lat  lookup_lng  \
0   0 sukhram bagh near model town railon road ropar   30.952042   76.523861   
1   01881 502433 c o mr sangat singh h no 4684 op...   30.973804   76.517214   
2                         02 shampura krishna colony   30.958224   76.507985   
3   1 canal rest house roper punjab nearby dc office   30.981542   76.517667   
4                              1 m head works colony   30.984002   76.521956   

           grid_id  
0  8a3d14716cb7fff  
1  8a3d14449aa7fff  
2  8a3d144494b7fff  
3  8a3d147324cffff  
4  8a3d14732637fff

In [18]:
address_data_path2 = '/home/uddeshya.singh/10.116.29.103:8000/TEST-DATA_wrong-pincodes.csv'

In [19]:
df2 = pd.read_csv(address_data_path2)

In [20]:
df2.head()

addr_hash  pincode  \
0  73a2f0b3a5207658befdbf271edd69e0   145026   
1  13312fdb5f6f9eeb92fab71d3893a624   145026   
2  347d55941433fb25ce66a4fbdd4e9831   145026   
3  aad817d99b682e74b3b43cbb8dc29593   145026   
4  d63dc9bdc8745ff09b4675b55e61f7b1   145026   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                      keshav vpo narot jaimal singh   32.296482   75.454434   
1  kharkhara tarf narot jaimal singh po narot jai...   32.297290   75.458707   
2              narot jaimal singh narot jaimal singh   32.296483   75.454431   
3                     pumma pumma narot jaimal singh   32.263652   75.429232   
4  village taloor po narrot jaimal singh narrot b...   32.316272   75.443284   

                            wrong_pin_pred  incorrect_pincode  \
0   [32.29658119819855, 75.45800207745341]             145025   
1   [32.296331586749204, 75.4550291754196]             145025   
2   [32.296331586749204, 75.4550291754196]             145025   
3  [32.264372017727524, 75.43006036929322]             145027   
4   [32.31671929629047, 75.44377692281842]             145025   

                           actual_pin_pred  wrong_pin_pred_drift  \
0   [32.29658119819855, 75.45800207745341]            336.277816   
1   [32.296331586749204, 75.4550291754196]            362.315290   
2   [32.296331586749204, 75.4550291754196]             58.786428   
3  [32.264372017727524, 75.43006036929322]            111.673604   
4   [32.31671929629047, 75.44377692281842]             67.921508   

   actual_pin_pred_drift  
0             336.277816  
1             362.315290  
2              58.786428  
3             111.673604  
4              67.921508

In [21]:
df2 = df2[['cleaned_address', 'lookup_lat', 'lookup_lng']]

In [22]:
df2.head()

cleaned_address  lookup_lat  lookup_lng
0                      keshav vpo narot jaimal singh   32.296482   75.454434
1  kharkhara tarf narot jaimal singh po narot jai...   32.297290   75.458707
2              narot jaimal singh narot jaimal singh   32.296483   75.454431
3                     pumma pumma narot jaimal singh   32.263652   75.429232
4  village taloor po narrot jaimal singh narrot b...   32.316272   75.443284

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [24]:
model = MultiTaskModel(model_dir, num_labels_res8, num_labels_res10, num_labels_res9)


Some weights of the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/PAN-India-MLM/pan_india_train_threshold_save_dir and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN thi

In [25]:
#model_checkpoint = "/home/uddeshya.singh/Fine-tuned-Model3/save/MM_model3.pt"
model_checkpoint = "/home/uddeshya.singh/Fine-tuned-Model6/save/MM_6model.pt"

#state_dict = torch.load(os.path.join(fine_tuned_lm_trained_save_dir, 'MM_model.pt'))
state_dict = torch.load(model_checkpoint)
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith('module.') else k  # remove 'module.' prefix
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

MultiTaskModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,

In [26]:
# print('Loading dataset...')
# test_df = pd.read_csv(address_data_path)
# print(list(test_df), len(test_df))

print('Loading dataset...')
test_df = df2
print(list(test_df), len(test_df))

# Load label mappings
def load_label_mappings(save_dir, resolution):
    save_id2label = os.path.join(save_dir, f'id2label_res{resolution}.json')
    with open(save_id2label, 'r') as f:
        id2label = json.load(f)
    
    save_label2id = os.path.join(save_dir, f'label2id_res{resolution}.json')
    with open(save_label2id, 'r') as f:
        label2id = json.load(f)
    
    return id2label, label2id

id2label_res8, label2id_res8 = load_label_mappings(fine_tuned_lm_trained_save_dir, 8)
id2label_res9, label2id_res9 = load_label_mappings(fine_tuned_lm_trained_save_dir, 9)
id2label_res10, label2id_res10 = load_label_mappings(fine_tuned_lm_trained_save_dir, 10)

Loading dataset...
['cleaned_address', 'lookup_lat', 'lookup_lng'] 180194


In [27]:
def tokenize(batch):
    return tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)

# Prepare the data
cleaned_address_list_test = test_df.cleaned_address.tolist()
fault_line_list = [ind for ind, a in enumerate(cleaned_address_list_test) if a is None or type(a) != str or pd.isnull(a)]
new_cleaned_address_list_test = [a for ind, a in enumerate(cleaned_address_list_test) if ind not in fault_line_list]

In [28]:
def predict(batch, head='None'):
    predictions = []
    for i in tqdm(range(0, len(batch), 128)):  # Using a batch size of 128 for prediction
        inputs = tokenizer(batch[i:i+128], return_tensors="pt", padding=True, truncation=True, max_length=100).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs[f'logits_{head}']
        batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(batch_predictions)
    return predictions

In [29]:
def process_labels(predictions, id2label):
    pred_grid_list = []
    for p in predictions:
        grid_id = id2label[str(p)]
        pred_grid_list.append(grid_id)
    return pred_grid_list

In [30]:
# Functions for calculating deviation and accuracy
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(vincenty((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred ", e)
        return None

def get_drift_for_row(row_local, head):
    pred_coor = get_centroid_of_grid(row_local[f'top_grid_{head}'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

In [31]:



def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in tqdm(drift_list_local, desc="Calculating accuracy"):
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue
        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1
    
    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]
    
    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))
    
    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local




In [12]:
# Evaluate function
def evaluate_resolution(resolution, id2label):
    print(f'Starting predictions for resolution {resolution}...')
    head = f'res{resolution}'
    
    initial_predictions = predict(new_cleaned_address_list_test, head=head)
    pred_grid_list = process_labels(initial_predictions, id2label)
    
    # Print the first 5 predictions
    print(f'First 5 predictions for resolution {resolution}: {pred_grid_list[:5]}')
    
    # Apply predictions and calculate accuracy
    test_df[f'top_grid_{head}'] = pred_grid_list
    test_df[f'pred_drift_{head}'] = test_df.apply(lambda row: get_drift_for_row(row, head), axis=1)
    test_drift_acc_dict = get_accuracy_for_df(test_df[f'pred_drift_{head}'].tolist())
    print(f'Accuracy for resolution {resolution}:')
    print(test_drift_acc_dict)


In [32]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1408/1408 [02:45<00:00,  8.52it/s]


First 5 predictions for resolution 10: ['8a3d32bad767fff', '8a3d32bad767fff', '8a3d32bad767fff', '8a3d32bae2b7fff', '8a3d32b1ec9ffff']


Calculating accuracy: 100%|██████████| 180194/180194 [00:00<00:00, 1510462.11it/s]

Accuracy for resolution 10:
{'num': {100: 142286, 200: 156800, 500: 165946, 1000: 169392, 2000: 171478, 'rest': 8716, 'None': 0}, 'per': {100: 0.7896267356293772, 200: 0.8701732577111335, 500: 0.9209296646947179, 1000: 0.9400534978967113, 2000: 0.9516299099859041, 'rest': 0.048370090014095916, 'None': 0.0}}


In [33]:
evaluate_resolution(9, id2label_res10)

Starting predictions for resolution 9...


100%|██████████| 1408/1408 [02:44<00:00,  8.53it/s]


First 5 predictions for resolution 9: ['8a3d16142467fff', '8a3d16142467fff', '8a3d16142467fff', '8a3d161424e7fff', '8a3d16065747fff']


Calculating accuracy: 100%|██████████| 180194/180194 [00:00<00:00, 1341060.93it/s]

Accuracy for resolution 9:
{'num': {100: 0, 200: 0, 500: 2, 1000: 12, 2000: 26, 'rest': 180168, 'None': 0}, 'per': {100: 0.0, 200: 0.0, 500: 1.109914869529507e-05, 1000: 6.659489217177043e-05, 2000: 0.00014428893303883593, 'rest': 0.9998557110669611, 'None': 0.0}}


In [13]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:48<00:00,  8.54it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d14449aa7fff', '8a3d144494a7fff', '8a3d147324cffff', '8a3d14732637fff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1480892.89it/s]

Accuracy for resolution 10:
{'num': {100: 142055, 200: 157895, 500: 167781, 1000: 171756, 2000: 174236, 'rest': 9863, 'None': 0}, 'per': {100: 0.77162287682171, 200: 0.8576635397259084, 500: 0.9113629080005866, 1000: 0.9329545516271137, 2000: 0.9464255645060539, 'rest': 0.05357443549394619, 'None': 0.0}}


In [14]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:48<00:00,  8.52it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d147324fffff', '893d147327bffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1465171.15it/s]

Accuracy for resolution 9:
{'num': {100: 43560, 200: 147766, 500: 168031, 1000: 172121, 2000: 174489, 'rest': 9610, 'None': 0}, 'per': {100: 0.2366118229865453, 200: 0.8026442294634952, 500: 0.9127208730085443, 1000: 0.9349371805387319, 2000: 0.947799825094107, 'rest': 0.05220017490589302, 'None': 0.0}}


In [15]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:48<00:00,  8.53it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1370862.64it/s]

Accuracy for resolution 8:
{'num': {100: 6887, 200: 25847, 500: 153286, 1000: 171584, 2000: 174268, 'rest': 9831, 'None': 0}, 'per': {100: 0.03740922003921803, 200: 0.1403972862427281, 500: 0.8326280968392007, 1000: 0.9320202717016388, 2000: 0.9465993840270724, 'rest': 0.05340061597292761, 'None': 0.0}}


In [21]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:48<00:00,  8.55it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1370449.03it/s]

Accuracy for resolution 8:
{'num': {100: 6911, 200: 25919, 500: 153485, 1000: 171730, 2000: 174361, 'rest': 9738, 'None': 0}, 'per': {100: 0.03753958467998197, 200: 0.1407883801650199, 500: 0.833709036985535, 1000: 0.9328133232662861, 2000: 0.9471045470100327, 'rest': 0.052895452989967354, 'None': 0.0}}


In [22]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:48<00:00,  8.52it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d147327bffff', '893d1473273ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1443926.58it/s]

Accuracy for resolution 9:
{'num': {100: 43626, 200: 147916, 500: 168258, 1000: 172333, 2000: 174569, 'rest': 9530, 'None': 0}, 'per': {100: 0.2369703257486461, 200: 0.8034590084682698, 500: 0.9139539052357699, 1000: 0.9360887348654799, 2000: 0.9482343738966534, 'rest': 0.05176562610334657, 'None': 0.0}}


In [23]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.50it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d14449aa7fff', '8a3d144494a7fff', '8a3d14449a0ffff', '8a3d1473271ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1483373.62it/s]

Accuracy for resolution 10:
{'num': {100: 141702, 200: 157949, 500: 167990, 1000: 171834, 2000: 174267, 'rest': 9832, 'None': 0}, 'per': {100: 0.7697054302304738, 200: 0.8579568601676272, 500: 0.9124981667472393, 1000: 0.9333782367095965, 2000: 0.9465939521670406, 'rest': 0.05340604783295944, 'None': 0.0}}


In [ ]:
## epoch 20

In [15]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:48<00:00,  8.54it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1368639.37it/s]

Accuracy for resolution 8:
{'num': {100: 6929, 200: 25975, 500: 153692, 1000: 171845, 2000: 174450, 'rest': 9649, 'None': 0}, 'per': {100: 0.03763735816055492, 200: 0.14109256432680242, 500: 0.8348334320121239, 1000: 0.9334379871699466, 2000: 0.9475879825528656, 'rest': 0.052412017447134425, 'None': 0.0}}


In [16]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:49<00:00,  8.49it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d147324fffff', '893d1473273ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1459212.63it/s]

Accuracy for resolution 9:
{'num': {100: 43548, 200: 147541, 500: 168165, 1000: 172255, 2000: 174598, 'rest': 9501, 'None': 0}, 'per': {100: 0.23654664066616332, 200: 0.8014220609563333, 500: 0.9134487422528096, 1000: 0.9356650497829971, 2000: 0.9483918978375765, 'rest': 0.05160810216242348, 'None': 0.0}}


In [17]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.48it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d14449aa7fff', '8a3d14449417fff', '8a3d14449b0ffff', '8a3d1473271ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1484642.80it/s]

Accuracy for resolution 10:
{'num': {100: 140958, 200: 157634, 500: 167881, 1000: 171803, 2000: 174305, 'rest': 9794, 'None': 0}, 'per': {100: 0.7656641263667918, 200: 0.8562458242576005, 500: 0.9119060940037698, 1000: 0.9332098490486097, 2000: 0.9468003628482501, 'rest': 0.05319963715174987, 'None': 0.0}}


In [13]:
### model epoch 19

In [14]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:48<00:00,  8.54it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1361474.68it/s]

Accuracy for resolution 8:
{'num': {100: 6945, 200: 25985, 500: 154428, 1000: 172174, 2000: 174671, 'rest': 9428, 'None': 0}, 'per': {100: 0.03772426792106421, 200: 0.14114688292712074, 500: 0.8388312809955513, 1000: 0.9352250691204189, 2000: 0.9487884236199001, 'rest': 0.051211576380099835, 'None': 0.0}}


In [15]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:49<00:00,  8.49it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d147324fffff', '893d1473273ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1434268.57it/s]

Accuracy for resolution 9:
{'num': {100: 43669, 200: 147793, 500: 168746, 1000: 172615, 2000: 174795, 'rest': 9304, 'None': 0}, 'per': {100: 0.23720389573001482, 200: 0.8027908896843546, 500: 0.9166046529313032, 1000: 0.9376205193944562, 2000: 0.9494619742638472, 'rest': 0.05053802573615283, 'None': 0.0}}


In [16]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.48it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d14449ab7fff', '8a3d144494a7fff', '8a3d1473271ffff', '8a3d1473271ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1484642.80it/s]

Accuracy for resolution 10:
{'num': {100: 140541, 200: 157891, 500: 168315, 1000: 172145, 2000: 174486, 'rest': 9613, 'None': 0}, 'per': {100: 0.7633990407335184, 200: 0.8576418122857811, 500: 0.9142635212575843, 1000: 0.9350675451794959, 2000: 0.9477835295140115, 'rest': 0.052216470485988514, 'None': 0.0}}


In [ ]:
### model epoch 15

In [15]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:48<00:00,  8.55it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1376026.98it/s]

Accuracy for resolution 8:
{'num': {100: 6941, 200: 26085, 500: 154677, 1000: 172603, 2000: 175018, 'rest': 9081, 'None': 0}, 'per': {100: 0.03770254048093689, 200: 0.1416900689303038, 500: 0.8401838141434772, 1000: 0.9375553370740742, 2000: 0.9506732790509455, 'rest': 0.04932672094905458, 'None': 0.0}}


In [16]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:49<00:00,  8.49it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d14449b7ffff', '893d1473273ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1466395.43it/s]

Accuracy for resolution 9:
{'num': {100: 43659, 200: 147203, 500: 168886, 1000: 172802, 2000: 175058, 'rest': 9041, 'None': 0}, 'per': {100: 0.23714957712969653, 200: 0.7995860922655745, 500: 0.9173651133357595, 1000: 0.9386362772204085, 2000: 0.9508905534522186, 'rest': 0.049109446547781356, 'None': 0.0}}


In [17]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.47it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d1444916ffff', '8a3d14449617fff', '8a3d147324cffff', '8a3d1473271ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1479301.30it/s]

Accuracy for resolution 10:
{'num': {100: 139080, 200: 157467, 500: 168303, 1000: 172204, 2000: 174655, 'rest': 9444, 'None': 0}, 'per': {100: 0.7554630932270138, 200: 0.8553387036322848, 500: 0.9141983389372023, 1000: 0.9353880249213739, 2000: 0.9487015138593908, 'rest': 0.051298486140609126, 'None': 0.0}}


In [18]:
#### model epoch 13

In [15]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:47<00:00,  8.58it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1378828.09it/s]

Accuracy for resolution 8:
{'num': {100: 6947, 200: 26060, 500: 154511, 1000: 172664, 2000: 174984, 'rest': 9115, 'None': 0}, 'per': {100: 0.03773513164112787, 200: 0.14155427242950805, 500: 0.8392821253781932, 1000: 0.937886680536016, 2000: 0.9504885958098632, 'rest': 0.04951140419013683, 'None': 0.0}}


In [16]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:49<00:00,  8.49it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d1444963ffff', '893d1473273ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1406530.89it/s]

Accuracy for resolution 9:
{'num': {100: 43625, 200: 146624, 500: 168849, 1000: 172850, 2000: 175003, 'rest': 9096, 'None': 0}, 'per': {100: 0.23696489388861428, 200: 0.7964410453071445, 500: 0.9171641345145818, 1000: 0.9388970065019364, 2000: 0.9505918011504679, 'rest': 0.04940819884953204, 'None': 0.0}}


In [17]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.48it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d14449aa7fff', '8a3d144494a7fff', '8a3d147324cffff', '8a3d1473271ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1500207.25it/s]

Accuracy for resolution 10:
{'num': {100: 137750, 200: 156938, 500: 168106, 1000: 172063, 2000: 174457, 'rest': 9642, 'None': 0}, 'per': {100: 0.7482387193846789, 200: 0.8524652496754463, 500: 0.9131282625109316, 1000: 0.9346221326568857, 2000: 0.9476260055730884, 'rest': 0.05237399442691161, 'None': 0.0}}


In [19]:
##model epoch 10

In [29]:
evaluate_resolution(8, id2label_res8)

Starting predictions for resolution 8...


100%|██████████| 1439/1439 [02:49<00:00,  8.50it/s]


First 5 predictions for resolution 8: ['883d14716dfffff', '883d14449bfffff', '883d144495fffff', '883d14449bfffff', '883d147327fffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1359011.40it/s]

Accuracy for resolution 8:
{'num': {100: 6970, 200: 26138, 500: 154379, 1000: 172745, 2000: 175083, 'rest': 9016, 'None': 0}, 'per': {100: 0.037860064421859976, 200: 0.14197795751199083, 500: 0.8385651198539916, 1000: 0.9383266611985942, 2000: 0.9510263499530144, 'rest': 0.04897365004698559, 'None': 0.0}}


In [31]:
evaluate_resolution(9, id2label_res9)

Starting predictions for resolution 9...


100%|██████████| 1439/1439 [02:49<00:00,  8.50it/s]


First 5 predictions for resolution 9: ['893d14716cbffff', '893d14449abffff', '893d144494bffff', '893d1444963ffff', '893d1444963ffff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1441573.24it/s]

Accuracy for resolution 9:
{'num': {100: 43382, 200: 145261, 500: 168807, 1000: 172968, 2000: 175105, 'rest': 8994, 'None': 0}, 'per': {100: 0.23564495190087942, 200: 0.7890374200837593, 500: 0.916935996393245, 1000: 0.9395379659856925, 2000: 0.9511458508737147, 'rest': 0.048854149126285315, 'None': 0.0}}


In [35]:
evaluate_resolution(10, id2label_res10)

Starting predictions for resolution 10...


100%|██████████| 1439/1439 [02:49<00:00,  8.48it/s]


First 5 predictions for resolution 10: ['8a3d14716cb7fff', '8a3d1444916ffff', '8a3d144494a7fff', '8a3d14449617fff', '8a3d14449617fff']


Calculating accuracy: 100%|██████████| 184099/184099 [00:00<00:00, 1477012.13it/s]

Accuracy for resolution 10:
{'num': {100: 134071, 200: 155910, 500: 168079, 1000: 172206, 2000: 174563, 'rest': 9536, 'None': 0}, 'per': {100: 0.7282549063275737, 200: 0.8468812975627245, 500: 0.9129816022900722, 1000: 0.9353988886414375, 2000: 0.9482017827364625, 'rest': 0.05179821726353755, 'None': 0.0}}
